In [0]:
#Importamos las librerias pyspark
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import *

In [0]:
spark.conf.set('fs.azure.account.key.datalakeintegrador.dfs.core.windows.net', 'al3gj4vaJIzYk6pNRm9FO48p82nXCRMkZMWUlcjdSIosycCLVqwtTiLbe8eR1Bm0VTUiytscWk7z+AStDSb3Fg==')

In [0]:
#Creamos los paths
#Para traer los datasets
trustedData="abfss://lucas-sarfati@datalakeintegrador.dfs.core.windows.net/trustedData"
gamesPath= trustedData + "/Tablas/games"
gamesDetailsPath= trustedData + "/Tablas/gamesDetail"
playersPath= trustedData + "/Tablas/players"
rankingPath= trustedData + "/Tablas/ranking"
teamsPath= trustedData + "/Tablas/teams"

In [0]:
#Traemos las tablas a los dataframes
games = spark.read.load(gamesPath, format='parquet', sep=',', header='true')
gamesDetails = spark.read.load(gamesDetailsPath, format='parquet', sep=',', header='true')
players = spark.read.load(playersPath, format='parquet', sep=',', header='true')
ranking = spark.read.load(rankingPath, format='parquet', sep=',', header='true')
teams = spark.read.load(teamsPath, format='parquet', sep=',', header='true')

In [0]:
#Unimos las tablas games con team para sacar el % de juegos ganados de local, de visita, la diferencia y el total
tablaPorcen=games.groupBy('TEAM_ID_home').agg(avg('HOME_TEAM_WINS').alias('% ganados local'),avg('AWAY_TEAM_WINS').alias('% perdidos local'))
auxiliar=games.groupBy('TEAM_ID_away').agg(avg('HOME_TEAM_WINS').alias('% perdidos visita'),avg('AWAY_TEAM_WINS').alias('% ganados visita'))
tablaPorcen=tablaPorcen.join(auxiliar,(tablaPorcen.TEAM_ID_home==auxiliar.TEAM_ID_away),'inner')
tablaPorcen=tablaPorcen.join(teams,(tablaPorcen.TEAM_ID_home==teams.TEAM_ID),'inner').orderBy('% ganados local',ascending=False)
tablaPorcen=tablaPorcen.drop('TEAM_ID_away')
tablaPorcen=tablaPorcen.drop('TEAM_ID')
tablaPorcen=tablaPorcen.drop('TEAM_ID_home')
tablaPorcen=tablaPorcen.drop('DLEAGUEAFFILIATION')
tablaPorcen=tablaPorcen.withColumn('% total ganado',(col('% ganados local')+col('% ganados visita'))/2)
tablaPorcen=tablaPorcen.withColumn('% diferencia',(col('% ganados local')-col('% ganados visita')))
tablaPorcen=tablaPorcen.drop('% perdidos visita')
tablaPorcen=tablaPorcen.drop('% perdidos local')
tablaPorcen=tablaPorcen.withColumn('% ganados local',col('% ganados local').cast(DecimalType(4, 2)))
tablaPorcen=tablaPorcen.withColumn('% ganados visita',col('% ganados visita').cast(DecimalType(4, 2)))
tablaPorcen=tablaPorcen.withColumn('% diferencia',col('% diferencia').cast(DecimalType(4, 2)))
tablaPorcen=tablaPorcen.withColumn('% total ganado',col('% total ganado').cast(DecimalType(4, 2)))
tablaPorcen.limit(10).display()

% ganados local,% ganados visita,NICKNAME,% total ganado,% diferencia
0.74,0.54,Spurs,0.64,0.20
0.68,0.44,Nuggets,0.56,0.24
0.67,0.46,Warriors,0.56,0.21
0.66,0.47,Heat,0.57,0.19
0.66,0.42,Jazz,0.54,0.24
0.65,0.49,Mavericks,0.57,0.16
0.65,0.48,Rockets,0.56,0.16
0.64,0.48,Celtics,0.56,0.16
0.63,0.41,Pacers,0.52,0.21
0.61,0.39,Trail Blazers,0.50,0.22


In [0]:
# agrupar por año el % o promedio de las estadisticas mas importantes
seasonStats=games.groupBy('SEASON').agg((avg('local Asistencias')+avg('visitante Asistencias'))/2,(avg('local % puntos de campo')+avg('visitante % tiros de campo'))/2,(avg('local % triples')+avg('visitante % triples'))/2,(avg('PTS_home')+avg('PTS_away'))/2,(avg('local Rebotes')+avg('visitante Rebotes'))/2).orderBy('SEASON',ascending=False)
# les cambiamos los nombres a las columnas
seasonStats=seasonStats.withColumnRenamed('((avg(local Asistencias) + avg(visitante Asistencias)) / 2)','Asistencias')
seasonStats=seasonStats.withColumnRenamed('((avg(local % puntos de campo) + avg(visitante % tiros de campo)) / 2)','% efectividad tiros de campo')
seasonStats=seasonStats.withColumnRenamed('((avg(local % triples) + avg(visitante % triples)) / 2)','% efectividad triples')
seasonStats=seasonStats.withColumnRenamed('((avg(PTS_home) + avg(PTS_away)) / 2)','Puntos')
seasonStats=seasonStats.withColumnRenamed('((avg(local Rebotes) + avg(visitante Rebotes)) / 2)','% efectividad rebotes')
seasonStats=seasonStats.withColumn("Asistencias",col('Asistencias').cast(DecimalType(5,2)))
seasonStats=seasonStats.withColumn("Puntos",col('Puntos').cast(DecimalType(5,2)))
seasonStats=seasonStats.withColumn("% efectividad tiros de campo", col("% efectividad tiros de campo").cast(DecimalType(3,2)))
seasonStats=seasonStats.withColumn("% efectividad triples", col("% efectividad triples").cast(DecimalType(3,2)))
seasonStats=seasonStats.withColumn("% efectividad rebotes", col('% efectividad rebotes').cast(DecimalType(5,2)))
seasonStats.display()

SEASON,Asistencias,% efectividad tiros de campo,% efectividad triples,Puntos,% efectividad rebotes
2021,24.35,0.46,0.35,109.67,44.82
2020,24.56,0.47,0.36,111.83,44.36
2019,24.25,0.46,0.36,111.37,44.82
2018,24.42,0.46,0.35,110.93,45.22
2017,23.14,0.46,0.36,106.07,43.50
2016,22.55,0.46,0.36,105.36,43.51
2015,22.04,0.45,0.35,102.24,43.82
2014,21.97,0.45,0.35,99.96,43.35
2013,21.79,0.45,0.36,100.50,42.69
2012,21.91,0.45,0.35,97.75,42.15


In [0]:
#Datos para la conexion con el datawarehouse
Server = 'integradorformacion.database.windows.net'
jdbcPort = 1433
jdbcDatabase = 'Refined'
jdbcUsername = 'administrador'
jdbcPassword = 'User1234'
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl = f"jdbc:sqlserver://{Server}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}"

% ganados local,% ganados visita,NICKNAME,% total ganado,% diferencia
0.74,0.54,Spurs,0.64,0.20
0.68,0.44,Nuggets,0.56,0.24
0.67,0.46,Warriors,0.56,0.21
0.66,0.47,Heat,0.57,0.19
0.66,0.42,Jazz,0.54,0.24
0.65,0.49,Mavericks,0.57,0.16
0.65,0.48,Rockets,0.56,0.16
0.64,0.48,Celtics,0.56,0.16
0.63,0.41,Pacers,0.52,0.21
0.61,0.39,Trail Blazers,0.50,0.22


In [0]:
#guardamos las tablas en el datawarehouse correspondiente

tablaPorcen.write.mode('overwrite')\
.format("jdbc")\
.option("driver", f"{jdbcDriver}")\
.option("url", f"{jdbcUrl}")\
.option('dbtable',"Sarfati.Localia")\
.save()

seasonStats.write.mode('overwrite')\
.format("jdbc")\
.option("url",f"jdbc:sqlserver://{Server}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("driver", f"{jdbcDriver}")\
.option('dbtable',"Sarfati.SeasonStats").save()
